# __Tweets Extraction: Brazil's Covid CPI (Parliamentary Commission of Inquiry)__

In [1]:
import itertools, datetime, os, itertools, json
from modules.hashtag_media import hashtag_media
import snscrape.modules.twitter as sntwitter
from modules.week import Week
import pandas as pd

In [2]:
def get_hashtags(hashtag_series):
    hashtag_list = []
    for hashtag_group in hashtag_series:
        if hashtag_group:
            for hashtag in hashtag_group:
                if hashtag not in hashtag_media:
                    hashtag_list.append('#'+hashtag)
    return hashtag_list

def get_hashtag_count(hashtag_list):
    hashtag_series = pd.Series(hashtag_list)
    return hashtag_series.unique().size

def get_unique_hashtags(hashtag_list):
    hashtag_series = pd.Series(hashtag_list)
    return hashtag_series.unique()

def get_top_10(hashtag_list):
    hashtag_series = pd.Series(hashtag_list)
    return hashtag_series.value_counts().head(10).index.values.tolist()


def day_tweet_extract(day):
    until = (datetime.datetime.strptime(day, '%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    query = f'cpi covid lang:pt since:{day} until:{until} -filter:nativeretweets'
    print('-Preliminary query extraction:')
    start_time = datetime.datetime.now()
    tweets = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(query).get_items(), 100000))
    runtime = datetime.datetime.now() - start_time
    print(f'--Runtime: {runtime}'
         f'\n--Tweets amount: {tweets.shape[0]}')
    hashtag_list = get_hashtags(tweets['hashtags'])
    hashtag_count = get_hashtag_count(hashtag_list)
    hashtag_top_10 = get_top_10(hashtag_list)
    print(f'--Unique hashtags amount: {hashtag_count}'
         f'\n--Hashtag Top 10: {hashtag_top_10}'
         '\n-Hashtag query extraction:')
    hashtag_query = ' OR '.join(hashtag_top_10)
    query = f'({hashtag_query}) lang:pt since:{day} until:{until}'
    start_time = datetime.datetime.now()
    tweets = pd.DataFrame(itertools.islice(sntwitter.TwitterSearchScraper(query).get_items(), 100000))
    runtime = datetime.datetime.now() - start_time
    print(f'--Runtime: {runtime}'
         f'\n--Tweets amount: {tweets.shape[0]}')
    hashtag_list = get_hashtags(tweets['hashtags'])
    hashtag_count = get_hashtag_count(hashtag_list)
    unique_hashtags = get_unique_hashtags(hashtag_list)
    hashtag_top_10 = get_top_10(hashtag_list)
    print(f'--Unique hashtags amount: {hashtag_count}'
         f'\n--Hashtag Top 10: {hashtag_top_10}\n')
    return hashtag_count, hashtag_top_10, unique_hashtags,tweets

def week_tweet_extract(week):
    week_dir = f'data/tweets_data/week_{week.number}'
    if not os.path.exists(week_dir):
        os.mkdir(week_dir)
    print(f'== EXTRACTING FROM WEEK {week.number}: SINCE {week.start} UNTIL {week.end} =='
         f'\n\nData is being stored in the following directory: {week_dir}\n')
    week_info = week.info
    week_hashtags = []
    week_tweets_amount = 0
    for idx, day in enumerate(week.days):
        _day = f'day_{idx+1}'
        if not os.path.isfile(f'data/tweets_data/week_{week.number}/{_day}_{day}.csv'):
            deponents = week_info['days_info'][_day]['deponents']
            print(f'Extracting tweets from Day {idx+1}: {day}'
                 f'\n-Deponents of the day: {deponents}')
            hashtag_count, hashtag_top_10, unique_hashtags,tweets = day_tweet_extract(day)
            week_info['days_info'][_day]['tweets_amount'] = tweets.shape[0]
            week_info['days_info'][_day]['hashtags_amount'] = hashtag_count
            week_info['days_info'][_day]['top_10_hashtags'] = hashtag_top_10
            week_tweets_amount += tweets.shape[0]
            for hashtag in unique_hashtags:
                week_hashtags.append(hashtag)
            tweets.to_csv(week_dir+'/'+_day+f'_{day}.csv')
    week_info['tweets_amount'] = week_tweets_amount
    week_info['hashtags_amount'] = get_hashtag_count(week_hashtags)
    week_info['top_10_hashtags'] = get_top_10(week_hashtags)
    print('Generating week info json file...\n')
    with open(week_dir + f'/week_{week.number}_info.json', 'w') as json_file:
        json.dump(week_info, json_file, indent=4)
        

In [3]:
deponents = {
    'day_3': ['Antonio Barra Torres'],
    'day_4': ['Fábio Wajngarten'],
    'day_5': ['Carlos Murillo']
}
week = Week(week_number=3,
           week_start='2021-05-09',
           deponents=deponents)

week_tweet_extract(week)

== EXTRACTING FROM WEEK 2: SINCE 2021-05-02 UNTIL 2021-05-08 ==

Data is being stored in the following directory: data/tweets_data/week_2

Extracting tweets from Day 1: 2021-05-02
-Deponents of the day: []
-Preliminary query extraction:
--Runtime: 0:00:59.627362
--Tweets amount: 1260
--Unique hashtags amount: 137
--Hashtag Top 10: ['#EuAutorizoPresidente', '#EuAutorizoBolsonaro', '#CPI', '#ForaBolsonaro', '#BolsonaroPresidenteAte2026', '#MoroSuspeitoSim', '#LulaLivre', '#ForaBolsonaroGenocida', '#BolsonaroOrgulhoDoBrasil', '#covid19']
-Hashtag query extraction:
--Runtime: 0:18:00.392619
--Tweets amount: 23035
--Unique hashtags amount: 2303
--Hashtag Top 10: ['#EuAutorizoPresidente', '#BolsonaroPresidenteAte2026', '#EuAutorizoBolsonaro', '#ForaBolsonaro', '#BolsonaroOrgulhoDoBrasil', '#ForaBolsonaroGenocida', '#EuAutorizoPresidenteBolsonaro', '#Bolsonaro2022', '#VotoImpressoAuditavelJa', '#LulaLivre']

Extracting tweets from Day 2: 2021-05-03
-Deponents of the day: []
-Preliminary query